# Setup


In [1]:
from pylabrobot.liquid_handling.backends.piper_backend import PiperBackend
import logging

back = PiperBackend(config={
  "dry": True,
  # "ws_address": "",
  # "ws_address": "ws://192.168.10.181:7125/websocket",
  "ws_address": "ws://10.1.0.4:7125/websocket",
  "sio_address": "http://10.1.0.4:3333",
  # "loglevel": 3,
  # "verbose": True
  "datatools": "mongo",
  "database": {
    "mongo_url": 'mongodb://10.1.0.4:27017/',
    "database_name": 'pipettin-2024-09-26'
  }
})

In [ ]:
from pylabrobot.resources.pipettin import SilverDeck

# Instantiate and empty SilverDeck.
deck = SilverDeck(
  workspace=back.database_tools.getWorkspaceByName("Experiment 2024-09-27"),
  platforms=back.database_tools.platforms,
  containers=back.database_tools.containers,
  tools=back.database_tools.tools,
  subset_platforms=False
)
deck.summary()

In [ ]:
from pylabrobot.liquid_handling import LiquidHandler

lh = LiquidHandler(backend=back, deck=deck)

await lh.setup()

In [ ]:
await back._home_machine()

# Send commands

In [ ]:
tip_rack = deck.get_resource("Blue tip rack")

pickups = await lh.pick_up_tips(tip_rack["A1"], use_channels=[1])

In [ ]:
tube_rack = deck["Tube Rack [5x16]"]

aspirations = await lh.aspirate(tube_rack["A1"], vols=[100], use_channels=[1])

In [ ]:
well_plate = deck["Standard 384-well plate"]

for well in well_plate["B2:B7"]:
  dispenses = await lh.dispense([well], vols=[15], use_channels=[1])

In [2]:
# TODO: Chequear que piper esté usando el tip que le pasé.

In [ ]:
await lh.stop()

In [ ]:
well_plate["B2:B7"]